In [27]:
class EnumTime:
    Monday = 0
    Tuesday = 1
    Wednesday = 2
    Thursday = 3
    Friday = 4
    Saturday = 5
    L1 = 0
    L2 = 1
    L3 = 2
    L4 = 3
    L5 = 4
    L6 = 5
    L7 = 6
    L8 = 7
    L9 = 8
    L10 = 9

class TimeFrame:
    # t0 and t1 is inclusive
    def __init__(self, t0, t1):
        self.t0 = t0
        self.t1 = t1

class Lesson:
    def __init__(self, name, dayOfWeek, timeFrame, address):
        self.Name = name
        self.TimeFrame = timeFrame
        self.DayOfWeek = dayOfWeek
        self.Address = address

    # isAvailable(Lessons{List}: `The current Timetable`)
    def isAvailable(self, Lessons):
        for i in range(self.TimeFrame.t0, self.TimeFrame.t1 + 1):
            if Lessons[i] != None:
                return False
        return True
    
    # regLesson(fullSchedule{FullSchedule}: `The full schedule`)
    # Overrides the time period to register
    def regLesson(self, fullSchedule):
        daySchedule = fullSchedule.Schedule[self.DayOfWeek]
        for i in range(self.TimeFrame.t0, self.TimeFrame.t1 + 1):
            daySchedule.Lessons[i] = self

    # unregLesson(fullSchedule{FullSchedule}: `The full schedule`)
    # Erases the lesson
    def unregLesson(self, fullSchedule):
        daySchedule = fullSchedule.Schedule[self.DayOfWeek]
        for i in range(self.TimeFrame.t0, self.TimeFrame.t1 + 1):
            if daySchedule.Lessons[i] == self:
                daySchedule.Lessons[i] = None


class DaySchedule:
    Lessons = None
    def __init__(self):
        self.Lessons = [None for i in range(11)]



class FullSchedule:
    Schedule = None
    def __init__(self):
        self.Schedule = [DaySchedule() for i in range(6)]
    
    def getLessonObject(self, weekday, time):
        return self.Schedule[weekday].Lessons[time]
    
    def destructObject(self):
        def getName(y):
            if y != None:
                return y.Name
            return ""
        return [[getName(y) for y in x.Lessons] for x in self.Schedule]

    # This function returns an artificial rating for how good a schedule is.
    def getScheduleRating(self):
        # Free Day Bonus
        def freeDayBonus():
            FREE_DAY_BONUS = 2000
            for d in schedule:
                if all(x is None for x in d):
                    return 0
            return FREE_DAY_BONUS

        # Seperated Lessons and Lesson Counts
        def timeBonus():
            SEPERATION_PENALTY = -50
            LESSON_COUNT_BONUS = 200
            LITTLE_LESSON_PENALTY = -500
            MANY_LESSON_PENALTY = -500
            NO_LUNCH_PENALTY = -15000
            LESSON_BONUS = 10000

            LESSON_RANGE = [3, 4]

            LUNCH_START = EnumTime.L5
            LUNCH_END = EnumTime.L7

            total = 0
            for d in schedule:
                empty_streak = -1
                lesson_count = 0
                have_lunch = False
                for i in range(len(d)):
                    if d[i] == None and empty_streak != -1:
                        empty_streak += 1
                    if d[i] != None:
                        # empty_streak is squared, to amplify the penalty for separated lessons
                        total += empty_streak * empty_streak * SEPERATION_PENALTY
                        lesson_count += 1
                    if LUNCH_START <= i <= LUNCH_END and d[i] == None:
                        have_lunch = True

                # lesson_count is squared, to amplify the reward for more lessons a day
                if 1 <= lesson_count <= LESSON_RANGE[0]:
                    dl = LESSON_RANGE[0] - lesson_count
                    total += dl * dl * LITTLE_LESSON_PENALTY
                elif lesson_count >= LESSON_RANGE[1]:
                    dl = lesson_count - LESSON_RANGE[1]
                    total += dl * dl * MANY_LESSON_PENALTY
                total += lesson_count * lesson_count * LESSON_COUNT_BONUS
                total += lesson_count * LESSON_BONUS
                
                

                # Lunch Penalty
                if not have_lunch:
                    total += NO_LUNCH_PENALTY
            return total
        
        # ...
        schedule = [x.Lessons for x in self.Schedule]
        rating = 10000
        
        rating += freeDayBonus()
        rating += timeBonus()
        
        return rating

class Course:
    AllLessons = None

    # lessons{List[Lesson]}
    def __init__(self, lessons):
        self.AllLessons = lessons
    
    def isAvailable(self, fullSchedule):
        for Lesson in self.AllLessons:
            dayOfWeekSchedule = fullSchedule.Schedule[Lesson.DayOfWeek]
            Lessons = dayOfWeekSchedule.Lessons
            if not Lesson.isAvailable(Lessons):
                return False
        return True
    
    # regCourse(fullSchedule{FullSchedule})
    # return False: Course not available
    # return True: Course successfully applied
    def regCourse(self, fullSchedule):
        if not self.isAvailable(fullSchedule):
            return False
        
        for Lesson in self.AllLessons:
            Lesson.regLesson(fullSchedule)
        return True
    
    # unregCourse(fullSchedule{FullSchedule})
    def unregCourse(self, fullSchedule):
        for Lesson in self.AllLessons:
            Lesson.unregLesson(fullSchedule)


# Using the Classes

## Example of how a course is defined:

```
ENGG1003AA = Course([
    Lesson("ENGG1003AA - LEC", EnumTime.Monday, TimeFrame(EnumTime.L1, EnumTime.L2), "Lady Shaw Bldg C1"),
    Lesson("ENGG1003AA - LAB", EnumTime.Monday, TimeFrame(EnumTime.L3, EnumTime.L3), "Lady Shaw Bldg C1")
])

ENGG1003AA.regCourse(fullSchedule)
```

In [28]:
# fullSchedule = FullSchedule()

# ENGG1003AA = Course([
#     Lesson("ENGG1003AA - LEC", EnumTime.Monday, TimeFrame(EnumTime.L1, EnumTime.L2), "Lady Shaw Bldg C1"),
#     Lesson("ENGG1003AA - LAB", EnumTime.Monday, TimeFrame(EnumTime.L3, EnumTime.L3), "Lady Shaw Bldg C1")
# ])

# ENGG1003AA.regCourse(fullSchedule)


## Example of how to get a lesson from the schedule:

```
print(fullSchedule.getLessonObject(EnumTime.Monday, EnumTime.L1).Name)
```

In [29]:
# print(fullSchedule.getLessonObject(EnumTime.Monday, EnumTime.L1).Name)

# Class Wishlist
In this section, we will be making a class wishlist, where the user can get to choose which classes they want to attend, and which to optimize for.

In [30]:
class ScheduleRanker:
    def __init__(self, maxData):
        self.MaxData = maxData
        self.Data = []
        self.RatingData = []

    def register_data(self, newData):
        rating = newData["rating"]
        if rating not in self.RatingData:
            self.RatingData.append(rating)
            self.RatingData.sort(reverse=True)

            self.Data.append(newData)
            self.Data.sort(key=lambda x: x["rating"], reverse=True)
            if len(self.Data) > self.MaxData:
                self.Data = self.Data[:self.MaxData]
    
    def get_ordered_data(self):
        return self.Data

class CourseChoices:
    def __init__(self, courses):
        self.Courses = courses
    
    def branchCourse(self, fullSchedule, courselist, i, l, scheduleRanker: ScheduleRanker, verbose=True):
        if i == l - 1:
            topscore = {"rating": 0, "layout": None}
            for course in self.Courses:
                success = course.regCourse(fullSchedule)
                if not success:
                    continue
                rating = fullSchedule.getScheduleRating()
                layout = fullSchedule.destructObject()
                course.unregCourse(fullSchedule)

                if rating > topscore["rating"]:
                    topscore = {"rating": rating, "layout": layout}

            return topscore
        
        # Branch here:
        topscore = {"rating": -1000000, "layout": None}

        for course in self.Courses:
            # Try to register to course
            success = course.regCourse(fullSchedule)
            if not success:
                pass
            # If course registered, go forward a branch
            output = courselist[i+1].branchCourse(fullSchedule, courselist, i+1, l, scheduleRanker=scheduleRanker, verbose=verbose)

            scheduleRanker.register_data(output)
            # Test if the output is top rating. If yes, then save it.
            if output["rating"] > topscore["rating"]:
                topscore = output
                if verbose:
                    print(topscore)
            course.unregCourse(fullSchedule)
        
        return topscore

In [31]:
class WishList:
    AllCourseChoice = list()
    def __init__(self, default = list()):
        self.AllCourseChoice = default
    
    def addCourseChoice(self, courseChoice):
        self.AllCourseChoice.append(courseChoice)
    
    def loadCourse(self, fullSchedule, maxData=100, verbose=True):
        clist = self.AllCourseChoice
        count = len(clist)
        scheduleRanker = ScheduleRanker(maxData=maxData)
        return scheduleRanker, clist[0].branchCourse(fullSchedule, clist, 0, count, scheduleRanker=scheduleRanker, verbose=verbose)


# Testing the program
Here, we steal CUTS's API to search for our program :D:D:D:D

現在，我們要當寄生蟲 :D:D

https://cuts.hk/ajax_planner2_get_course.php?year=2023&term=1&key=KEYWORDHERE&mode=code

In [32]:
# CourseList = ["ENGG1110", "AIST1000", "MATH1510", "PHYS1003", "ENGG1003EB", "CHLT1001", "UGFH1000"]

# YEAR = 2023
# TERM = 1

In [33]:
import requests
def get_api(courseList, year, term, verbose):
    def api(search_term):
        return f"https://cuts.hk/ajax_planner2_get_course.php?year={year}&term={term}&key={search_term}&mode=code"

    dayconvert = {
        "M": 0,
        "T": 1,
        "W": 2,
        "H": 3,
        "F": 4,
        "S": 5
    }

    all_courses = []
    for course in courseList:
        data = requests.get(api(course)).json()
        if verbose:
            print(data)
        courses = []
        for coursedata in data["courses"]:
            lessons = []
            timecodes = []
            for perioddata in coursedata["periods"]:
                day = perioddata['day']
                start = perioddata['start']
                end = perioddata['end']


                timecode = f"{day}{start}{end}"
                if timecode in timecodes:
                    pass
                lessons.append(Lesson(
                    name=f"{coursedata['coursecode']} ({perioddata['type']})",
                    dayOfWeek=dayconvert[day],
                    timeFrame=TimeFrame(
                        t0=start,
                        t1=end,
                    ),
                    address=perioddata['venue']
                ))
            courses.append(Course(lessons))
        all_courses.append(CourseChoices(courses))
    return all_courses

        

In [34]:
def optimizeCourses(courseList, maxData, year, term, verbose=False):
    wishlist = WishList(get_api(courseList, year, term, verbose=verbose))

    schedule = FullSchedule()
    return wishlist.loadCourse(schedule, maxData=maxData, verbose=verbose)

In [35]:


def giveCodes(json):
    output = ""
    subjects = []
    for x in json["layout"]:
        for y in x:
            if y != "":
                subjects.append(y[0:(y.index(" "))])
    output += f"Rating: {json['rating']}"
    # insert the list to the set
    list_set = set(subjects)
    # convert the set to the list
    unique_list = (list(list_set))
    for x in unique_list:
        output += "\n" + x

    return output

In [36]:
import pandas as pd
import numpy as np
import datetime

def showCourseTimetable(scheduleInfo, saveExcel=False, askBeforeSave=True):
    # Define the days of the week
    days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

    # Define your 2D array
    array_2d = scheduleInfo["layout"]

    # Transpose the array
    transposed_array = np.transpose(array_2d)

    # Create a DataFrame from the transposed array
    df = pd.DataFrame(transposed_array, columns=days_of_week)

    # Create a styler object for the DataFrame
    styler = df.style

    # Define a function to apply color coding
    def color_code(val):
        if val.startswith("CSCI"):
            return "background-color: red"
        if val != "":
            return 'background-color: green'
        else:
            return ''

    # Apply the color coding to the DataFrame styler
    styled_df = styler.applymap(color_code)

    display(styled_df)

    ct = datetime.datetime.now()

    # Check if we want to save
    save = saveExcel
    if askBeforeSave:
        isSave = input("Do you want to save this timetable? (y/n)")
        if isSave.lower() == "y":
            save = True
    # Save the table to an Excel file
    if save:
        styled_df.to_excel(f"output/result_{ct.year}-{ct.month}-{ct.day}+{ct.hour}_{ct.minute}_{ct.second}.xlsx", index=False)

In [37]:
def printAndOptimizeCourses(courseList, year, term, maxData=100, courseListVerbose=False, showCoursesVerbose=False, saveExcel=False, askBeforeSave=False):

    scheduleRanker, best = optimizeCourses(
        courseList = courseList,
        year = year,
        term = term,
        maxData = maxData,
        verbose=courseListVerbose
    )
    json = giveCodes(best)
    if showCoursesVerbose:
        print(json)

    showCourseTimetable(best, saveExcel=saveExcel, askBeforeSave=askBeforeSave)

    return scheduleRanker

In [38]:
scheduleRanker = printAndOptimizeCourses(
        courseList = [
                "ENGG1110",
                "AIST1000",
                "MATH1510",
                "PHYS1003",
                "ENGG1003EB",
                "CHLT1001",
                "UGFH1000",
                "PHED1017"
            ],
        year = 2023,
        term = 1,
        maxData = 100,
        courseListVerbose=False,
        showCoursesVerbose=True,
        saveExcel=False,
        askBeforeSave=False
    )

rank = scheduleRanker.get_ordered_data()
for i in range(0, 5):
    showCourseTimetable(rank[i], askBeforeSave=False)

Rating: 332799
ENGG1110A
ENGG1003EB
PHYS1003A
PHED1017G
UGFH1000U
CHLT1001HX
AIST1000
MATH1510G


,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
0,,,,,,
1,,,,PHYS1003A (TUT/MP1/04),CHLT1001HX (LEC/MP1/01),
2,PHYS1003A (EXR/MP3/04),,UGFH1000U (TUT/MP1/03),PHYS1003A (LEC/MP4/01),CHLT1001HX (LEC/MP1/01),
3,PHYS1003A (LEC/MP3/01),AIST1000 (LEC/MP1/01),UGFH1000U (TUT/MP1/03),,CHLT1001HX (LEC/MP1/01),
4,PHYS1003A (LEC/MP3/01),AIST1000 (PRJ/MP1/01),PHED1017G (LEC/MP1/01),ENGG1110A (LEC/MP1/01),,
5,,,PHED1017G (LEC/MP1/01),ENGG1110A (LAB/MP1/01),,
6,MATH1510G (TUT/MP3/01),UGFH1000U (TUT/MP1/01),,,,
7,ENGG1110A (LEC/MP4/01),UGFH1000U (TUT/MP1/01),ENGG1003EB (LEC/MP1/01),,UGFH1000U (LEC/MP1/01),
8,ENGG1110A (LEC/MP4/01),,ENGG1003EB (LEC/MP1/01),,,
9,MATH1510G (LEC/MP4/01),UGFH1000U (TUT/MP1/02),ENGG1003EB (LAB/MP1/01),MATH1510G (LEC/MP1/01),,


,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
0,,,,,,
1,,,,PHYS1003A (TUT/MP1/04),CHLT1001HX (LEC/MP1/01),
2,PHYS1003A (EXR/MP3/04),,UGFH1000U (TUT/MP1/03),PHYS1003A (LEC/MP4/01),CHLT1001HX (LEC/MP1/01),
3,PHYS1003A (LEC/MP3/01),AIST1000 (LEC/MP1/01),UGFH1000U (TUT/MP1/03),,CHLT1001HX (LEC/MP1/01),
4,PHYS1003A (LEC/MP3/01),AIST1000 (PRJ/MP1/01),PHED1017G (LEC/MP1/01),ENGG1110A (LEC/MP1/01),,
5,,,PHED1017G (LEC/MP1/01),ENGG1110A (LAB/MP1/01),,
6,MATH1510G (TUT/MP3/01),UGFH1000U (TUT/MP1/01),,,,
7,ENGG1110A (LEC/MP4/01),UGFH1000U (TUT/MP1/01),ENGG1003EB (LEC/MP1/01),,UGFH1000U (LEC/MP1/01),
8,ENGG1110A (LEC/MP4/01),,ENGG1003EB (LEC/MP1/01),,,
9,MATH1510G (LEC/MP4/01),UGFH1000U (TUT/MP1/02),ENGG1003EB (LAB/MP1/01),MATH1510G (LEC/MP1/01),,


,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
0,,,,,,
1,,,,PHYS1003A (TUT/MP1/04),CHLT1001HZ (LEC/MP1/01),
2,PHYS1003A (EXR/MP3/04),,UGFH1000U (TUT/MP1/03),PHYS1003A (LEC/MP4/01),CHLT1001HZ (LEC/MP1/01),
3,PHYS1003A (LEC/MP3/01),AIST1000 (LEC/MP1/01),UGFH1000U (TUT/MP1/03),,CHLT1001HZ (LEC/MP1/01),
4,PHYS1003A (LEC/MP3/01),AIST1000 (PRJ/MP1/01),PHED1017G (LEC/MP1/01),ENGG1110A (LEC/MP1/01),,
5,,,PHED1017G (LEC/MP1/01),ENGG1110A (LAB/MP1/01),,
6,MATH1510F (TUT/MP3/01),UGFH1000U (TUT/MP1/01),,,,
7,ENGG1110A (LEC/MP4/01),UGFH1000U (TUT/MP1/01),ENGG1003EB (LEC/MP1/01),,UGFH1000U (LEC/MP1/01),
8,ENGG1110A (LEC/MP4/01),,ENGG1003EB (LEC/MP1/01),,,
9,MATH1510F (LEC/MP3/01),UGFH1000U (TUT/MP1/02),ENGG1003EB (LAB/MP1/01),MATH1510F (LEC/MP4/01),,


,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
0,,,,,,
1,,,,PHYS1003A (TUT/MP1/04),CHLT1001JD (LEC/MP1/01),
2,PHYS1003A (EXR/MP3/04),,UGFH1000U (TUT/MP1/03),PHYS1003A (LEC/MP4/01),CHLT1001JD (LEC/MP1/01),
3,PHYS1003A (LEC/MP3/01),AIST1000 (LEC/MP1/01),UGFH1000U (TUT/MP1/03),,CHLT1001JD (LEC/MP1/01),
4,PHYS1003A (LEC/MP3/01),AIST1000 (PRJ/MP1/01),PHED1017G (LEC/MP1/01),,,
5,,,PHED1017G (LEC/MP1/01),,,
6,MATH1510H (TUT/MP3/01),UGFH1000U (TUT/MP1/01),,,,
7,ENGG1110B (LEC/MP4/01),UGFH1000U (TUT/MP1/01),ENGG1003EB (LEC/MP1/01),ENGG1110B (LEC/MP1/01),UGFH1000U (LEC/MP1/01),
8,ENGG1110B (LEC/MP4/01),,ENGG1003EB (LEC/MP1/01),ENGG1110B (LAB/MP1/01),,
9,MATH1510H (LEC/MP4/01),UGFH1000U (TUT/MP1/02),ENGG1003EB (LAB/MP1/01),MATH1510H (LEC/MP1/01),,


,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
0,,,,,,
1,,,,PHYS1003A (TUT/MP1/04),CHLT1001HC (LEC/MP1/01),
2,PHYS1003A (EXR/MP3/04),,UGFH1000U (TUT/MP1/03),PHYS1003A (LEC/MP4/01),CHLT1001HC (LEC/MP1/01),
3,PHYS1003A (LEC/MP3/01),AIST1000 (LEC/MP1/01),UGFH1000U (TUT/MP1/03),,CHLT1001HC (LEC/MP1/01),
4,PHYS1003A (LEC/MP3/01),AIST1000 (PRJ/MP1/01),PHED1017G (LEC/MP1/01),ENGG1110A (LEC/MP1/01),,
5,,,PHED1017G (LEC/MP1/01),ENGG1110A (LAB/MP1/01),,
6,MATH1510F (TUT/MP3/01),UGFH1000U (TUT/MP1/01),,,,
7,ENGG1110A (LEC/MP4/01),UGFH1000U (TUT/MP1/01),ENGG1003EB (LEC/MP1/01),,UGFH1000U (LEC/MP1/01),
8,ENGG1110A (LEC/MP4/01),,ENGG1003EB (LEC/MP1/01),,,
9,MATH1510F (LEC/MP3/01),UGFH1000U (TUT/MP1/02),ENGG1003EB (LAB/MP1/01),MATH1510F (LEC/MP4/01),,


,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
0,,,,,,
1,,,,PHYS1003A (TUT/MP1/04),CHLT1001JY (LEC/MP1/01),
2,PHYS1003A (EXR/MP3/04),,UGFH1000U (TUT/MP1/03),PHYS1003A (LEC/MP4/01),CHLT1001JY (LEC/MP1/01),
3,PHYS1003A (LEC/MP3/01),AIST1000 (LEC/MP1/01),UGFH1000U (TUT/MP1/03),,CHLT1001JY (LEC/MP1/01),
4,PHYS1003A (LEC/MP3/01),AIST1000 (PRJ/MP1/01),PHED1017G (LEC/MP1/01),,,
5,,,PHED1017G (LEC/MP1/01),,,
6,MATH1510F (TUT/MP3/01),UGFH1000U (TUT/MP1/01),,,,
7,ENGG1110B (LEC/MP4/01),UGFH1000U (TUT/MP1/01),ENGG1003EB (LEC/MP1/01),ENGG1110B (LEC/MP1/01),UGFH1000U (LEC/MP1/01),
8,ENGG1110B (LEC/MP4/01),,ENGG1003EB (LEC/MP1/01),ENGG1110B (LAB/MP1/01),,
9,MATH1510F (LEC/MP3/01),UGFH1000U (TUT/MP1/02),ENGG1003EB (LAB/MP1/01),MATH1510F (LEC/MP4/01),,


In [39]:
# scheduleRanker = printAndOptimizeCourses(
#         courseList = [
#                 "ENGG1120",
#                 "ENGG1130",
#                 "AIST1110",
#                 "ELTU1001",
#                 ""
#             ],
#         year = 2023,
#         term = 2,
#         courseListVerbose=False,
#         showCoursesVerbose=False,
#         saveExcel=False,
#         askBeforeSave=False
#     )
